### Run Multiple Scenario on Pool

In [2]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅Connected, local lib:0.46.5, server:0.50.1

#### Pool

In `Pool` , it will have a field named `assets` ,which contains a list of `assets`

In [3]:
mortgage1 = ["Mortgage"
            ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 40
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 30
            ,"status": "current"}]

mortgage2 = ["Mortgage"
            ,{"originBalance": 800.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 25
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 600.0
            ,"currentRate": 0.075
            ,"remainTerm": 15
            ,"status": "current"}]

myPool = {'assets':[ mortgage1, mortgage2],
         'cutoffDate':"2022-03-01"}

#### Set Multi Assumption

It is just a map!

In [4]:
multiAssump= \
    {"cdr001":("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
    ,"non":("Pool",("Mortgage",None,None,None,None)
                    ,None
                    ,None
                    )}

Make sure call `runPoolByScenarios`

In [5]:
r = localAPI.runPoolByScenarios(myPool,poolAssump=multiAssump,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Use keys from `multiAssump` to retrive results

In [6]:
r['cdr001']['PoolConsol']['flow'].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9542.79,345.80,61.80,0,7.62,0,7.62,0.075,None,None,345.80,0,0,7.62,0,7.62
2022-04-01,9186.98,347.68,59.58,0,8.13,0,8.13,0.075,None,None,693.48,0,0,15.75,0,15.75
2022-05-01,8829.85,349.56,57.36,0,7.57,0,7.57,0.075,None,None,1043.04,0,0,23.32,0,23.32
2022-06-01,8470.88,351.45,55.13,0,7.52,0,7.52,0.075,None,None,1394.49,0,0,30.84,0,30.84
2022-07-01,8110.54,353.35,52.89,0,6.99,0,6.99,0.075,None,None,1747.84,0,0,37.83,0,37.83


#### Run Multiple Pools with Multiple Assumptions

In [7]:
receivable1 = ["Invoice"
                  ,{"start":"2024-04-01","originBalance":2000
                    ,"originAdvance":1500,"dueDate":"2024-06-01"}
                  ,{"status":"Current"}]

myMixedPool = {"invoicePool": {'assets':[receivable1],}
              ,"mortgagePool": {'assets':[mortgage1]}
              ,'cutoffDate':"2022-03-01"}

In [8]:
mPoolAssump0 = ("Pool",
               ("Mortgage",None,None,None,None)
                ,None
                ,None)

mPoolAssump = ("Pool",
               ("Mortgage",{"CDR":0.01},None,None,None)
                ,None
                ,None)

iPoolAssump0 = ("Pool"
                ,("Receivable", None, None, None)
                ,None
                ,None)

iPoolAssump = ("Pool"
                ,("Receivable", {"CDR":0.01}, None, None)
                ,None
                ,None)


r = localAPI.runPoolByScenarios(myMixedPool
                  ,poolAssump = {"NonStress":("ByPoolId",
                                    {"invoicePool":iPoolAssump0
                                    ,"mortgagePool":mPoolAssump0})
                                 ,"Stress":("ByPoolId",
                                    {"invoicePool":iPoolAssump
                                    ,"mortgagePool":mPoolAssump})}
                  ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [9]:
r['NonStress']['invoicePool']['flow'].head()

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-04-01,2000,0,0,0,0,0,0,0,0,0,0,0,0
2024-06-01,0,0,2000,0,0,0,0,0,0,0,0,0,0


In [10]:
r['Stress']['mortgagePool']['flow'].head()

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2022-03-01,9059.53,307.14,58.54,0,7.22,0,7.22,0.075,None,None,307.14,0,0,7.22,0,7.22
2022-04-01,8743.01,308.80,56.57,0,7.72,0,7.72,0.075,None,None,615.94,0,0,14.94,0,14.94
2022-05-01,8425.32,310.48,54.59,0,7.21,0,7.21,0.075,None,None,926.42,0,0,22.15,0,22.15
2022-06-01,8105.99,312.15,52.61,0,7.18,0,7.18,0.075,None,None,1238.57,0,0,29.33,0,29.33
2022-07-01,7785.46,313.84,50.62,0,6.69,0,6.69,0.075,None,None,1552.41,0,0,36.02,0,36.02
